In [2]:
library(Seurat)
library(SeuratWrappers)
library(Matrix)
library(glmGamPoi)
library(sctransform)
library(zinbwave)
library(dplyr)

Attaching SeuratObject

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds,

In [3]:
exp_data <- read.csv('../../data/muraro/matrix.csv', row.names=1)
exp_data_meta <- read.csv('../../data/muraro/meta.csv', row.names=1)[1:2285,]
colnames(exp_data) <- rownames(exp_data_meta)
size_factor <- exp_data_meta$nCount_RNA/mean(exp_data_meta$nCount_RNA)
pbmc <- CreateSeuratObject(exp_data)
pbmc <- NormalizeData(pbmc)
pbmc <- FindVariableFeatures(pbmc, selection.method='vst', nfeatures=2000)
pbmc <- ScaleData(pbmc, features = rownames(pbmc))

Centering and scaling data matrix



In [4]:
donor <- read.csv('/data01/hanbin973/batch/batch_labels.csv')$X0
pbmc@meta.data$donor <- donor[1:2285]
pbmc@meta.data$logsf <- log(size_factor)

# PCA

In [7]:
pbmc_pca <- RunPCA(pbmc)

PC_ 1 
Positive:  IFITM3, ZFP36L1, REST, MYH9, S100A11, SOX4, NOTCH2, MYL12A, RBPMS, LGALS3 
	   PMEPA1, AHNAK, CAV2, TMSB4X, YAP1, EPS8, CLIC4, PPIC, ANO6, TMSB10 
	   TPM1, CDC42EP1, SH3BP4, MSN, RHOC, LITAF, CSDA, LAMC1, ANXA2, OSMR 
Negative:  SCG3, SYT7, PAM, ERO1LB, ABCC8, GAD2, VGF, MIR7-3HG, PLCXD3, LOC728392 
	   KIF5C, MLXIPL, STX1A, G6PC2, STMN2, PEG10, CFC1, RGS4, SCD, WNT4 
	   RASD1, C14orf132, ELAVL4, PRUNE2, KCNK16, SYT13, PCP4, CRYBA2, ATP2A3, MEIS2 
PC_ 2 
Positive:  CD24, ELF3, TACSTD2, KRT8, KIAA1522, KRT18, CFB, ANXA4, OCLN, KRT7 
	   CLDN4, SERPINA3, TMC5, RASEF, PDZK1IP1, CLDN1, TC2N, ABCC3, SDC4, TMPRSS2 
	   ERBB3, CXADR, LAD1, TM4SF1, CLMN, GATM, LCN2, KCNQ1, ATP10B, NR5A2 
Negative:  SPARC, PDGFRB, COL6A3, MRC2, COL5A2, COL5A1, CDH11, NID1, COL5A3, COL1A2 
	   LAMA4, COL15A1, COL6A2, COL3A1, FBN1, BGN, LOXL2, THBS2, LUM, LTBP2 
	   SFRP2, PRRX1, CYGB, VCAN, SRPX2, WNT5A, EDNRA, ITGA11, COL12A1, COL4A1 
PC_ 3 
Positive:  CFTR, ONECUT2, DCDC2, TINAGL1, ALDH1A3,

In [8]:
write.csv(Embeddings(pbmc_pca), 'pca.csv')

# SCTransform

In [9]:
pbmc_sct <- SCTransform(pbmc, batch_var='donor', method='glmGamPoi')

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 17138 by 2285

Model formula is y ~ (log_umi) : donor + donor + 0

Get Negative Binomial regression parameters per gene

Using 2000 genes, 2285 cells



  |======================================================================| 100%


Found 88 outliers - those will be ignored in fitting/regularization step


Some genes not detected in batch D28 -- assuming a low mean.

Some genes not detected in batch D29 -- assuming a low mean.

Some genes not detected in batch D30 -- assuming a low mean.

Some genes not detected in batch D31 -- assuming a low mean.

Second step: Get residuals using fitted parameters for 17138 genes



  |======================================================================| 100%


Computing corrected count matrix for 17138 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 1.135273 mins

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix

Set default assay to SCT



In [10]:
pbmc_sct <- RunPCA(pbmc_sct)
write.csv(Embeddings(pbmc_sct), 'sct.csv')

PC_ 1 
Positive:  CHGB, CHGA, CPE, SCG5, SCG2, PCSK2, SLC30A8, VGF, TM4SF4, TTR 
	   GCG, GC, ALDH1A1, SCGN, G6PC2, MAFB, PAX6, NEUROD1, BEX1, PPP1R1A 
	   RGS4, SCG3, PTPRN, ABCC8, ENPP2, SLC7A2, CRYBA2, LOXL4, TMEM176B, PAM 
Negative:  CD24, TACSTD2, ZFP36L1, IFITM3, TMSB4X, CD44, ANXA4, SOX4, SERPINA3, ANXA2 
	   SAT1, TM4SF1, CFB, KRT7, TPM1, S100A11, GDF15, PMEPA1, CAV2, CLDN1 
	   SOD2, SLC4A4, KRT8, KRT18, NFIB, CFTR, BACE2, CXCL2, MYH9, GSTA1 
PC_ 2 
Positive:  CD24, SERPINA3, SPINK1, REG1A, CFB, TACSTD2, GSTA1, CTRB2, REG1B, PRSS1 
	   CPA1, ANXA4, PRSS3P2, GSTA2, PLA2G1B, PNLIP, OLFM4, CPA2, DUOX2, CPB1 
	   MUC1, C15orf48, CTRB1, GDF15, SAT1, TM4SF1, ANPEP, ALDOB, LYZ, REG3A 
Negative:  SPARC, COL4A1, PXDN, COL4A2, UNC5B, IGFBP4, COL1A2, LGALS1, COL5A2, SERPINH1 
	   LAMA4, TIMP1, COL5A1, COL6A1, COL15A1, COL1A1, COL6A2, COL3A1, NID1, PDGFRB 
	   COL6A3, SERPINE1, FSTL1, CALD1, MRC2, FN1, COL5A3, LOXL2, NOTCH3, ITGA1 
PC_ 3 
Positive:  TM4SF4, GC, GCG, PCSK2, TTR, CHGB, CFTR

# GLM-PCA

In [11]:
pbmc_gpca <- RunGLMPCA(pbmc, L=30, 
                       X=as.matrix(model.matrix(as.formula('~factor(donor)'), pbmc@meta.data)), 
                       sz=size_factor)

Sparse matrices are not supported for minibatch='none'. Coercing to dense matrix. If this exhausts memory, consider setting minibatch to 'stochastic' or 'memoized'.



In [12]:
write.csv(Embeddings(pbmc_gpca, reduction='glmpca'), 'glmpca.csv')

# ZINB-WaVE

In [26]:
sceobj <- as.SingleCellExperiment(pbmc)
filter1 <- rowSums(assay(sceobj)>5)>5
sceobj <- sceobj[filter1,]
assay(sceobj) %>% log1p %>% rowVars -> vars
names(vars) <- rownames(sceobj)
vars <- sort(vars, decreasing = TRUE)
sceobj <- sceobj[names(vars)[1:2000],]
assay(sceobj) <- apply(assay(sceobj), c(1,2), function(x){
        as.integer(x)
})
sceobj <- zinbsurf(sceobj, K=20, X='~factor(donor)+logsf')

In [35]:
reducedDim(sceobj)

ERROR: Error in .get_internal_missing(x, basefun = reducedDim, namefun = reducedDimNames, : no available entries for 'reducedDim(<SingleCellExperiment>, ...)'


# ARI & Silhouette

In [5]:
library(mclust)
library(cluster)

Package 'mclust' version 5.4.7
Type 'citation("mclust")' for citing this R package in publications.



In [6]:
pca <- read.csv('pca.csv', row.names=1)
sct <- read.csv('sct.csv', row.names=1)
glmpca <- read.csv('glmpca.csv', row.names=1)
rdpca <- read.csv('rdpca.csv', header=FALSE)
rownames(rdpca) <- rownames(pca)
zinbwave <- read.csv('zinbwave.csv', row.names=1)

pbmc[['pca']] <- CreateDimReducObject(embeddings = as.matrix(pca), key = "pc_", assay='RNA')
pbmc[['sct']] <- CreateDimReducObject(embeddings = as.matrix(sct), key = "sct_", assay='RNA')
pbmc[['glmpca']] <- CreateDimReducObject(embeddings = as.matrix(glmpca), key = "glmpc_", assay='RNA')
pbmc[['rdpca']] <- CreateDimReducObject(embeddings = as.matrix(rdpca), key = "rdpc_", assay='RNA')
pbmc[['zinbwave']] <- CreateDimReducObject(embeddings = as.matrix(zinbwave), key = "zinbwave_", assay='RNA')

In [8]:
celltype <- exp_data_meta$celltype
intname <- 1:length(unique(celltype))
names(intname) <- unique(celltype)

ress <- c(0.4, 0.5, 0.9, 1.5, 2)
methods <- c('pca', 'sct', 'glmpca', 'rdpca', 'zinbwave')
aris <- data.frame(matrix(1:(length(ress)*length(methods)), nrow=length(methods)))
rownames(aris) <- methods
colnames(aris) <- ress
for (res in ress){
    for (method in methods){
        obj <- FindNeighbors(pbmc, reduction=method, dims=1:30, verbose=FALSE)
        obj <- FindClusters(obj, resolution=res, verbose=FALSE)
        
        aris[method, as.character(res)] <- adjustedRandIndex(obj$seurat_clusters, celltype)
    }
}

In [9]:
sils <- data.frame(matrix(1:length(methods), nrow=length(methods)))
rownames(sils) <- methods
colnames(sils) <- c('Silhouette')
for (method in methods){
    obj <- RunTSNE(pbmc, reduction=method, perplexity=70, dims=1:30)
    dist_mat <- dist(Embeddings(obj, reduction='tsne'))
    silscore <- silhouette(as.vector(intname[celltype]), dist_mat) 
    
    
    sils[method, 'Silhouette']<- (summary(silscore)$avg.width)

}

In [10]:
df <- cbind(aris,sils)
colnames(df) <- c(paste('ARI_', ress), 'Silhouette')
df

,ARI_ 0.4,ARI_ 0.5,ARI_ 0.9,ARI_ 1.5,ARI_ 2,Silhouette
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
pca,0.7500075,0.6961619,0.57034592,0.48211451,0.38499285,0.4634273
sct,0.6035467,0.5749250,0.57492498,0.50795812,0.37234576,0.4386040
glmpca,0.1067382,0.1015778,0.08342266,0.06228432,0.05907963,-0.1059929
rdpca,0.9529520,0.9529520,0.75447917,0.58491420,0.43794636,0.6106239
zinbwave,0.9407659,0.9407659,0.94076592,0.69575713,0.59828659,0.4765458
